Bitswan Gitops Webhook Pipeline
-------------------------------

This pipeline allows you to easilly deploy Bitswan pipelines to any server. You just need to POST to the webhook in your CI and your pipelines will be running in no time.

To install bitswan you need to initialize a git repository on your server/the place you want to run your bitswan installation and run the bitswan-pre docker image with the following ENV vars set:

- `BS_WEBHOOK_PORT` - defaults to 8080
- `BS_WEBHOOK_SECRET` - A secret that will be added to your webhook URL. To deploy post `{"action": "deploy-git"}` to the URL `https://localhost:<BS_WEBHOOK_PORT>/?secret=<BS_WEBHOOK_SECRET>`
- `BS_BITSWAN_DIR` - The directory where your `bitswan.yaml` file resides. Should be in a checked out git repository. Defaults to `/mnt/repo/bitswan`.

Simply adding the following curl command to your CI/CD pipeline should be enough to automatically deploy your data machines:

```
curl -X POST "https://<WEBHOOK_URL>:<BS_WEBHOOK_PORT>/?secret=<BS_WEBHOOK_SECRET>" -d '{"action": "deploy-git"}'
```

In [12]:
from bspump.jupyter import *
import bspump.http.web.source
import bspump.common
import time
import json
import os
import asyncio
import yaml
import datetime

In [2]:
new_pipeline("BitswanGitopsWebHookPipeline")

In [3]:
@register_source
def source(app, pipeline):
   return bspump.http.web.source.WebHookSource(
       app,
       pipeline,
       config = {
           "port": int(os.environ.get("BS_WEBHOOK_PORT", 8080)),
           "path": "/",
           "secret_qparam": os.environ.get("BS_WEBHOOK_SECRET", "not-secure")
       })

In [4]:
sample_events([
    b"""{"action": "deploy-git"}"""
])

In [5]:
@step
def parse_json(event):
    return json.loads(event)

{'action': 'deploy-git'}


In [6]:
@step
def get_bitswan_dir(event):
    return os.environ.get("BS_BITSWAN_DIR", "/mnt/repo/bitswan")

/mnt/repo/bitswan


In [13]:
sample_events(["/home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo"])

In [14]:
@async_step
async def git_pull(inject, bitswan_dir):
    print("Pulling in new changes.")
    git_repo_root = os.path.abspath(bitswan_dir)  # Convert to absolute path

    while git_repo_root != os.path.dirname(git_repo_root):  # Check until the root of the file system
        if os.path.isdir(os.path.join(git_repo_root, '.git')):
            break  # Return the path if .git directory is found
        git_repo_root = os.path.dirname(git_repo_root)  # Move up one directory level

    await asyncio.create_subprocess_exec("git", "config", "--global", "--add", "safe.directory", git_repo_root) 
    await asyncio.create_subprocess_exec("git", "config", "pull.rebase", "false", cwd=bitswan_dir) 
    await asyncio.create_subprocess_exec("git", "pull", cwd=bitswan_dir)
    print("Done pulling in changes.")
    await inject(bitswan_dir)

Pulling in new changes.
Done pulling in changes.
/home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo


In [15]:
@step
def load_bitswan_yaml(bitswan_dir):
    dmy = yaml.full_load(open(os.path.join(bitswan_dir, "bitswan.yaml")))
    return (bitswan_dir, dmy)

('/home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo', {'env-dir': '/etc/bitswan-secrets/', 'default-networks': ['default'], 'data-machines': {'demo-zabbix-demo-loader': {'volumes': ['/mnt/data/zabbix-demo/:/data/'], 'source': 'zabbix-demo-loader'}, 'demo-zabbix-cleaner': {'source': 'zabbix-cleaner'}, 'demo-zabbix-analyzer': {'source': 'zabbix-analyzer'}, 'demo-zabbix-shoveler': {'source': 'zabbix-shoveler'}}})
Already up to date.


In [17]:
@step
def generate_docker_compose(event):
    (bitswan_dir, bitswan_yaml) = event
    dc = {
        "version": "3",
        "services": {}
    }
    for deployment_id, conf in bitswan_yaml["data-machines"].items():
        entry = {}
        entry["environment"] = {"DEPLOYMENT_ID": deployment_id}
        if "env-dir" in bitswan_yaml:
           env_file = os.path.join(bitswan_yaml["env-dir"], deployment_id)
           if os.path.exists(env_file):
               entry["env_file"] = [env_file]
        if "default-networks" in bitswan_yaml:
            entry["networks"] = bitswan_yaml["default-networks"].copy()
        dc["services"][deployment_id] = entry
        data_machine_dir = os.path.join(bitswan_dir, conf["source"])
        app_yaml = yaml.full_load(open(os.path.join(data_machine_dir, "app.yaml")))
        dockerfile_path = app_yaml["dockerfile"]
        entry["build"] = {
            "dockerfile": os.path.join(data_machine_dir, dockerfile_path),
            "context": bitswan_dir,
            "args": {
                "DATA_MACHINE_SOURCE_PATH": conf["source"],
            }
        }
        passthroughs = ["volumes", "network_mode", "ports", "restart", "devices", ]
        for passthrough in passthroughs:
            if passthrough in conf:
              entry[passthrough] = conf[passthrough]
    dc_yaml = yaml.dump(dc)
    print(dc_yaml)
    return (bitswan_dir, dc_yaml)

services:
  demo-zabbix-analyzer:
    build:
      args:
        DATA_MACHINE_SOURCE_PATH: zabbix-analyzer
      context: /home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo
      dockerfile: /home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo/zabbix-analyzer/Dockerfile
    environment:
      DEPLOYMENT_ID: demo-zabbix-analyzer
    networks:
    - default
  demo-zabbix-cleaner:
    build:
      args:
        DATA_MACHINE_SOURCE_PATH: zabbix-cleaner
      context: /home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo
      dockerfile: /home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo/zabbix-cleaner/Dockerfile
    environment:
      DEPLOYMENT_ID: demo-zabbix-cleaner
    networks:
    - default
  demo-zabbix-demo-loader:
    build:


In [ ]:
@async_step
async def docker_compose_up_daemon(inject, event):
    (bitswan_dir, docker_compose_yaml) = event
    cmd = ["docker-compose", "-f", "/dev/stdin", "build", "--pull"]

    # Create a subprocess with stdin pipe
    proc = await asyncio.create_subprocess_exec(
        *cmd, 
        stdin=asyncio.subprocess.PIPE, 
        stdout=asyncio.subprocess.PIPE, 
        stderr=asyncio.subprocess.PIPE,
        cwd=bitswan_dir
    )

    # Send docker_compose_yaml as input to the process and wait for completion
    stdout, stderr = await proc.communicate(input=docker_compose_yaml.encode())

    build_result = {
        "cmd": cmd,
        "stdout": stdout.decode("utf-8"),
        "stderr": stderr.decode("utf-8"),
        "returncode": proc.returncode,
    }
    
    cmd = ["docker-compose", "-f", "/dev/stdin", "up", "-d"]

    # Create a subprocess with stdin pipe
    proc = await asyncio.create_subprocess_exec(
        *cmd, 
        stdin=asyncio.subprocess.PIPE, 
        stdout=asyncio.subprocess.PIPE, 
        stderr=asyncio.subprocess.PIPE,
        cwd=bitswan_dir
    )

    # Send docker_compose_yaml as input to the process and wait for completion
    stdout, stderr = await proc.communicate(input=docker_compose_yaml.encode())

    up_result = {
        "cmd": cmd,
        "stdout": stdout.decode("utf-8"),
        "stderr": stderr.decode("utf-8"),
        "returncode": proc.returncode,
    }
    event = {
        "@timestamp": datetime.datetime.now().timestamp(),
        "local-time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "build": build_result,
        "up": up_result
    }
    await inject(event)

In [ ]:
@step
def serialize_yaml(event):
    return yaml.dump(event)

In [ ]:
@register_sink
def init_sink(app, pipeline):
    return bspump.common.PrintSink(app, pipeline)

In [ ]:
end_pipeline()